# Ejercicio 2 — Laberinto (Punto 2)

En este notebook se resuelve el problema del laberinto con búsqueda informada usando una cola de prioridad y heurística de Manhattan.

Incluye:
- Implementación base (resolución del laberinto).
- Análisis del efecto de cambiar la función de costo.
- Manejo de múltiples salidas.
- Prueba con un laberinto más grande y nuevos obstáculos, y discusión de limitaciones.



In [1]:
import heapq
import itertools
from typing import List, Tuple, Dict, Optional, Callable

Position = Tuple[int, int]
Action = str  # 'Up', 'Down', 'Left', 'Right'

class Node:
    def __init__(self, position: Position, parent: Optional['Node']=None, path_cost: float=0.0, action: Optional[Action]=None):
        self.position = position
        self.parent = parent
        self.path_cost = path_cost
        self.action = action

    def __lt__(self, other: 'Node'):
        # No depender de esta comparación en el heap; se usa un desempate explícito.
        return self.path_cost < other.path_cost

class Problem:
    def __init__(self, maze: List[List[str]], start: Position, goals: List[Position],
                 move_cost: Callable[[Position, Position, Action], float],
                 actions: Optional[Dict[Tuple[int, int], Action]] = None):
        self.maze = maze
        self.start = start
        self.goals = set(goals)
        self.rows = len(maze)
        self.cols = len(maze[0]) if self.rows > 0 else 0
        # Movimientos por defecto: vectores y nombres
        self.moves: Dict[Tuple[int, int], Action] = actions or {
            (-1, 0): 'Up',
            (1, 0): 'Down',
            (0, -1): 'Left',
            (0, 1): 'Right',
        }
        self.move_cost = move_cost

    def in_bounds(self, pos: Position) -> bool:
        r, c = pos
        return 0 <= r < self.rows and 0 <= c < self.cols

    def is_blocked(self, pos: Position) -> bool:
        r, c = pos
        return self.maze[r][c] == '#'

    def is_goal(self, pos: Position) -> bool:
        return pos in self.goals

    def get_neighbors(self, pos: Position) -> List[Tuple[Position, Action]]:
        neighbors: List[Tuple[Position, Action]] = []
        for (dr, dc), action in self.moves.items():
            nr, nc = pos[0] + dr, pos[1] + dc
            npos = (nr, nc)
            if self.in_bounds(npos) and not self.is_blocked(npos):
                neighbors.append((npos, action))
        return neighbors

def manhattan(a: Position, b: Position) -> int:
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def reconstruct_path(node: Node) -> List[Tuple[Position, Optional[Action]]]:
    path: List[Tuple[Position, Optional[Action]]] = []
    while node is not None:
        path.append((node.position, node.action))
        node = node.parent
    path.reverse()
    return path

def min_heuristic_to_goals(pos: Position, goals: List[Position], heuristic: Callable[[Position, Position], float]) -> float:
    return min(heuristic(pos, g) for g in goals)

def compute_path_cost(path: List[Tuple[Position, Optional[Action]]], move_cost: Callable[[Position, Position, Action], float]) -> float:
    total = 0.0
    for i in range(1, len(path)):
        prev_pos = path[i-1][0]
        pos, action = path[i]
        if action is None:
            continue
        total += move_cost(prev_pos, pos, action)
    return total

def informed_search(maze: List[List[str]], start: Position, goals: List[Position],
                    heuristic: Callable[[Position, Position], float],
                    move_cost: Callable[[Position, Position, Action], float],
                    a_star: bool = True) -> Optional[List[Tuple[Position, Optional[Action]]]]:
    """
    Si a_star=True, usa f(n) = g(n) + h(n) (A*). Si False, usa solo h(n) (Greedy Best-First).
    Para múltiples metas usa h(n) = min_g h(n, g).
    """
    problem = Problem(maze, start, goals, move_cost)

    start_node = Node(start, path_cost=0.0, action=None, parent=None)
    # prioridad: f = g + h (o solo h si greedy). Usar un contador para desempate estable.
    counter = itertools.count()
    h0 = min_heuristic_to_goals(start, list(problem.goals), heuristic)
    start_priority = (start_node.path_cost + h0) if a_star else h0
    frontier: List[Tuple[float, int, Node]] = [(start_priority, next(counter), start_node)]
    heapq.heapify(frontier)

    reached: Dict[Position, float] = {start: 0.0}

    while frontier:
        _, _, node = heapq.heappop(frontier)
        if problem.is_goal(node.position):
            return reconstruct_path(node)

        for neighbor_pos, action in problem.get_neighbors(node.position):
            step_cost = move_cost(node.position, neighbor_pos, action)
            new_cost = node.path_cost + step_cost
            if neighbor_pos not in reached or new_cost < reached[neighbor_pos]:
                reached[neighbor_pos] = new_cost
                neighbor_node = Node(neighbor_pos, parent=node, path_cost=new_cost, action=action)
                h = min_heuristic_to_goals(neighbor_pos, list(problem.goals), heuristic)
                priority = (new_cost + h) if a_star else h
                heapq.heappush(frontier, (priority, next(counter), neighbor_node))

    return None


In [2]:
# Laberinto base del enunciado
maze = [
    ["#", "#", "#", "#", "#", "#", "#","#"],
    ["#", "S", "#", " ", "#", " ", "E","#"],
    ["#", " ", " ", " ", "#", " ", " ","#"],
    ["#", " ", "#", " ", " ", " ", "#","#"],
    ["#", "#", "#", "#", "#", "#", "#","#"],
    ["#", "#", "#", "#", "#", "#", "#","#"]
]

# Ubicar S y E
start = None
goals = []
for r, row in enumerate(maze):
    for c, val in enumerate(row):
        if val == 'S':
            start = (r, c)
        if val == 'E':
            goals.append((r, c))

assert start is not None and goals, "No se encontraron S o E en el laberinto"

# Costo uniforme por paso
uniform_cost = lambda a, b, action: 1.0

path = informed_search(maze, start, goals, heuristic=manhattan, move_cost=uniform_cost, a_star=True)
path


[((1, 1), None),
 ((2, 1), 'Down'),
 ((2, 2), 'Right'),
 ((2, 3), 'Right'),
 ((3, 3), 'Down'),
 ((3, 4), 'Right'),
 ((3, 5), 'Right'),
 ((2, 5), 'Up'),
 ((1, 5), 'Up'),
 ((1, 6), 'Right')]

In [3]:
def pretty_print_path(maze, path):
    # clonar
    drawn = [row[:] for row in maze]
    for (r, c), action in path[1:-1]:
        if drawn[r][c] == ' ':
            drawn[r][c] = '.'
    return '\n'.join(''.join(row) for row in drawn)

print(pretty_print_path(maze, path))
print("Acciones:", [a for (_, a) in path if a is not None])


########
#S# #.E#
#...#. #
# #...##
########
########
Acciones: ['Down', 'Right', 'Right', 'Down', 'Right', 'Right', 'Up', 'Up', 'Right']


## 2) ¿Cómo cambia el comportamiento si cambiamos la función de costo?

A continuación comparamos:
- Costo uniforme por paso (1 por movimiento).
- Costo que penaliza moverse verticalmente (por ejemplo, `Up`/`Down` valen 2 y `Left`/`Right` valen 1).

Esto afecta la ruta elegida por A*, ya que `g(n)` cambia y el algoritmo buscará caminos que minimicen el costo acumulado, no sólo la cantidad de pasos.


In [4]:
# Costo no uniforme: penaliza vertical
penalized_cost = lambda a, b, action: 2.0 if action in ('Up', 'Down') else 1.0

path_uniform = informed_search(maze, start, goals, heuristic=manhattan, move_cost=uniform_cost, a_star=True)
path_penalized = informed_search(maze, start, goals, heuristic=manhattan, move_cost=penalized_cost, a_star=True)

print('Ruta con costo uniforme:')
print(pretty_print_path(maze, path_uniform))
print('Costo total:', compute_path_cost(path_uniform, uniform_cost) if path_uniform else None)
print('Acciones:', [a for (_, a) in path_uniform if a is not None] if path_uniform else None)

print('\nRuta con costo penalizado (vertical=2):')
print(pretty_print_path(maze, path_penalized))
print('Costo total:', compute_path_cost(path_penalized, penalized_cost) if path_penalized else None)
print('Acciones:', [a for (_, a) in path_penalized if a is not None] if path_penalized else None)


Ruta con costo uniforme:
########
#S# #.E#
#...#. #
# #...##
########
########
Costo total: 9.0
Acciones: ['Down', 'Right', 'Right', 'Down', 'Right', 'Right', 'Up', 'Up', 'Right']

Ruta con costo penalizado (vertical=2):
########
#S# # E#
#...#..#
# #...##
########
########
Costo total: 13.0
Acciones: ['Down', 'Right', 'Right', 'Down', 'Right', 'Right', 'Up', 'Right', 'Up']


## 3) Múltiples salidas

La implementación ya acepta una lista de metas `goals`. La prioridad se calcula con respecto a la meta más cercana (por Manhattan) desde el nodo actual.

Ejemplo: añadimos otra `E` y observamos que el algoritmo puede llegar a cualquiera de ellas, priorizando la más cercana según heurística.


In [5]:
maze_multi = [row[:] for row in maze]
maze_multi[2][6] = 'E'  # nueva salida

# Recalcular metas
goals_multi = []
for r, row in enumerate(maze_multi):
    for c, val in enumerate(row):
        if val == 'E':
            goals_multi.append((r, c))

path_multi = informed_search(maze_multi, start, goals_multi, heuristic=manhattan, move_cost=uniform_cost, a_star=True)
print(pretty_print_path(maze_multi, path_multi))
print('Acciones:', [a for (_, a) in path_multi if a is not None])


########
#S# # E#
#...#.E#
# #...##
########
########
Acciones: ['Down', 'Right', 'Right', 'Down', 'Right', 'Right', 'Up', 'Right']


## 4) Laberinto más grande y nuevos obstáculos

Agregamos celdas con un obstáculo adicional: por ejemplo, agua `~` que tiene costo 3 por paso (pero no bloquea), y paredes `#` que bloquean. Mostramos la ruta y discutimos limitaciones: tamaño del espacio de estados, consistencia de la heurística, y posibles trade-offs.



In [6]:
# Laberinto más grande con agua (~)
maze_big_rows = [
    "#####################",
    "#S   ~     #     E  #",
    "# ### ### ### ### ##",
    "#   #   ~   #     ##",
    "### # ##### # ### ##",
    "#   #     #   #   ##",
    "# ### ### ##### ###",
    "#   ~   #     ~  ##",
    "##### # ### #### ##",
    "#     #   ~   #  ##",
    "#####################",
]
max_width = max(len(r) for r in maze_big_rows)
maze_big = [list(r.ljust(max_width)) for r in maze_big_rows]

# localizar S y E
start_big = None
goals_big = []
for r, row in enumerate(maze_big):
    for c, val in enumerate(row):
        if val == 'S':
            start_big = (r, c)
        if val == 'E':
            goals_big.append((r, c))

assert start_big is not None and goals_big

# costo: 1 para espacio, 3 para agua, bloqueado para '#'
def terrain_cost(a: Position, b: Position, action: Action) -> float:
    r, c = b
    cell = maze_big[r][c]
    if cell == '~':
        return 3.0
    return 1.0

path_big = informed_search(maze_big, start_big, goals_big, heuristic=manhattan, move_cost=terrain_cost, a_star=True)
print('Longitud del camino:', len(path_big) if path_big else None)
print(pretty_print_path(maze_big, path_big) if path_big else 'No hay camino')
print('Costo total:', compute_path_cost(path_big, lambda a,b,act: terrain_cost(a,b,act)))


Longitud del camino: 25
#####################
#S...~.... # ....E  #
# ### ###.###.### ## 
#   #   ~...#.    ## 
### # #####.#.### ## 
#   #     #...#   ## 
# ### ### ##### ###  
#   ~   #     ~  ##  
##### # ### #### ##  
#     #   ~   #  ##  
#####################
Costo total: 26.0


### Comentarios sobre limitaciones
- Con laberintos más grandes, el número de estados crece y A* puede requerir mucha memoria/tiempo si la heurística no es muy informativa.
- La heurística Manhattan es admisible y consistente cuando solo hay movimientos ortogonales y costos uniformes. Si hay costos variables (p. ej. agua), sigue siendo admisible si no sobrestima el costo real más bajo; puede volverse poco informativa, aumentando la exploración.
- Obstáculos con alto costo pero transitables (como `~`) pueden hacer que A* explore muchos desvíos; considerar heurísticas ponderadas o infladas, o variantes como D* para entornos con costos dinámicos.
- Para múltiples metas, priorizar por la más cercana es efectivo, pero si los costos del terreno varían mucho, una heurística mejor sería la distancia ponderada estimada (difícil sin conocimiento previo).



## Comparación A* vs Greedy Best-First

Comparamos A* (usa costo acumulado + heurística) contra Greedy Best-First (solo heurística) en el laberinto base con costo uniforme, para observar diferencias en costo y secuencia de acciones.


In [7]:
path_astar = informed_search(maze, start, goals, heuristic=manhattan, move_cost=uniform_cost, a_star=True)
path_greedy = informed_search(maze, start, goals, heuristic=manhattan, move_cost=uniform_cost, a_star=False)

print('A* (g+h) — costo, pasos y acciones:')
print(pretty_print_path(maze, path_astar))
print('Costo:', compute_path_cost(path_astar, uniform_cost))
print('Pasos:', len(path_astar)-1)
print('Acciones:', [a for (_, a) in path_astar if a is not None])

print('\nGreedy Best-First (h) — costo, pasos y acciones:')
print(pretty_print_path(maze, path_greedy))
print('Costo:', compute_path_cost(path_greedy, uniform_cost))
print('Pasos:', len(path_greedy)-1)
print('Acciones:', [a for (_, a) in path_greedy if a is not None])


A* (g+h) — costo, pasos y acciones:
########
#S# #.E#
#...#. #
# #...##
########
########
Costo: 9.0
Pasos: 9
Acciones: ['Down', 'Right', 'Right', 'Down', 'Right', 'Right', 'Up', 'Up', 'Right']

Greedy Best-First (h) — costo, pasos y acciones:
########
#S# #.E#
#...#. #
# #...##
########
########
Costo: 9.0
Pasos: 9
Acciones: ['Down', 'Right', 'Right', 'Down', 'Right', 'Right', 'Up', 'Up', 'Right']
